<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5f10498b2252223ea8ac284c0c6369d9d4cb16f870f3f54cd44cb6d99d4251d4
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-23 13:45:10
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.42 C
-------------------
Today PnL: 86.50 K (0.61%)
Current PnL: -19.19 L (-12.78%)
CY Booked + Current PnL: -7.09 L (-4.72%)
-------------------
Total profit:  2.32 L
Total loss:  -21.51 L
-------------------
Total Booked + Current PnL: 19.89 L (16.16%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.49%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 86.38 L (60.63%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.61%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.0,-8.78,73.0,X-LC,3.06,228587.0,18221.0,10561.0,0.01,8.66,4.62,13.68,37.0,1.73,1.61,26.62,XY25,NTT,REFINERIES
85,VOLTAS,1530.0,2.56,59.0,X-MC,2.40,215250.0,23508.0,14250.0,-1.43,12.26,6.62,19.69,99.0,1.65,1.52,19.89,XY25,NTT,AC
2,ABBOTINDIA,35195.0,-8.83,45.0,X-MC,3.49,90060.0,-618.0,15526.0,-0.78,-0.68,17.24,16.44,101.0,-0.04,0.63,19.36,X40,ATH,PHARMA
78,TTKPRESTIG,770.0,96.80,51.0,M-SC,1.90,84770.0,-16007.0,16098.0,0.37,-15.88,18.99,0.09,245.0,-0.99,0.60,11.37,OX40N,NTT,DURABLES
42,ITC,452.0,-37.08,63.0,X-LC,2.14,203351.0,3213.0,17671.0,0.73,1.61,8.69,10.44,4.0,0.18,1.43,7.26,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BSOFT,836.99,-13.63,63.0,H-SC,15.14,97257.0,-43396.0,116173.0,8.65,-30.85,119.45,51.74,131.0,-0.37,0.69,12.52,XR,ATH,IT
82,VAIBHAVGBL,521.00,54.82,60.0,H-SC,4.59,134006.0,-48769.0,159842.0,5.62,-26.68,119.28,60.77,125.0,-0.31,0.94,20.02,XR,NTT,JEWELLERY
40,INFY,2275.00,-15.65,61.0,X-LC,7.34,328318.0,15430.0,156247.0,4.69,4.93,47.59,54.87,3.0,0.10,2.31,11.91,X40,BTT,IT
12,BANKINDIA,190.00,-19.25,73.0,H-MC,13.67,208691.0,28883.0,83539.0,4.17,16.06,40.03,62.52,88.0,0.35,1.47,55.63,XR,NTT,BANKS
68,SONACOMS,806.63,-32.46,70.0,M-SC,10.22,83991.0,-17168.0,57173.0,3.90,-16.97,68.07,39.54,202.0,-0.30,0.59,18.44,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-41.63,46.0,H-SC,18.19,88410.0,-14090.0,54690.0,-2.68,-13.75,61.86,39.61,114.0,-0.26,0.62,7.98,AR,ATH,MISC
72,TANLA,1963.11,-28.95,34.0,H-SC,17.07,190194.0,-80567.0,410515.0,-2.50,-29.76,215.84,121.86,133.0,-0.20,1.34,48.79,AR,ATH,IT
46,KPIGREEN,731.05,4.41,61.0,H-SC,5.31,124375.0,-922.0,59849.0,-2.29,-0.74,48.12,47.03,141.0,-0.02,0.88,55.73,MH,ATH,POWER
71,SYMPHONY,1306.00,-43.99,48.0,M-SC,18.64,119223.0,-51918.0,51862.0,-1.79,-30.34,43.50,-0.03,196.0,-1.00,0.84,3.94,OX40N,NTT,DURABLES
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,26.64,81216.0,-32333.0,72388.0,-1.74,-28.47,89.13,35.27,268.0,-0.45,0.57,98.78,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,41.14,47.0,M-MC,10.28,223206.0,-1756.0,169413.0,-0.25,-0.78,75.90,74.53,192.0,-0.01,1.57,31.22,XY24,BTT,STEEL
46,KPIGREEN,731.05,4.41,61.0,H-SC,5.31,124375.0,-922.0,59849.0,-2.29,-0.74,48.12,47.03,141.0,-0.02,0.88,55.73,MH,ATH,POWER
37,INDIAMART,4810.62,-55.39,42.0,H-SC,7.36,124497.0,1161.0,130460.0,-1.17,0.94,104.79,106.72,119.0,0.01,0.88,24.54,AR,ATH,MISC
87,WIPRO,420.00,-13.34,50.0,M-LC,6.25,152334.0,1389.0,108066.0,1.80,0.92,70.94,72.51,53.0,0.01,1.07,7.00,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,122.87,69.0,H-SC,14.44,140508.0,-1971.0,21498.0,3.58,-1.38,15.30,13.70,163.0,-0.09,0.99,65.84,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.69,61.0,H-SC,2.19,229311.0,-40356.0,76682.0,-0.80,-14.97,33.44,13.47,138.0,-0.53,1.62,16.60,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,96.80,51.0,M-SC,1.90,84770.0,-16007.0,16098.0,0.37,-15.88,18.99,0.09,245.0,-0.99,0.60,11.37,OX40N,NTT,DURABLES
67,SIS,528.0,2082.38,54.0,H-SC,3.42,88064.0,-22968.0,46577.0,0.23,-20.69,52.89,21.26,156.0,-0.49,0.62,18.98,OX40N,NTT,MISC
73,TATAELXSI,9161.0,-23.04,52.0,H-MC,7.60,104063.0,-27772.0,69993.0,2.29,-21.07,67.26,32.03,98.0,-0.40,0.73,16.56,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,47.87,51.0,M-LC,5.85,206828.0,3658.0,38470.0,-0.29,1.80,18.60,20.74,55.0,0.10,1.46,7.44,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.80,53.0,M-LC,10.67,211845.0,6769.0,105795.0,0.65,3.30,49.94,54.89,52.0,0.06,1.49,28.19,X5K,ATH,METALS
80,UNIONBANK,163.00,-10.08,62.0,M-LC,9.02,163182.0,22342.0,22144.0,1.31,15.86,13.57,31.59,66.0,1.01,1.15,46.55,XY24,NTT,BANKS
87,WIPRO,420.00,-13.34,50.0,M-LC,6.25,152334.0,1389.0,108066.0,1.80,0.92,70.94,72.51,53.0,0.01,1.07,7.00,XR,NTT,IT
12,BANKINDIA,190.00,-19.25,73.0,H-MC,13.67,208691.0,28883.0,83539.0,4.17,16.06,40.03,62.52,88.0,0.35,1.47,55.63,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-19.25,73.0,H-MC,13.67,208691.0,28883.0,83539.0,4.17,16.06,40.03,62.52,88.0,0.35,1.47,55.63,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-38.27,67.0,M-SC,3.99,104280.0,12782.0,65968.0,-0.51,13.97,63.26,86.07,223.0,0.19,0.73,51.43,XR,NTT,DURABLES
32,HINDZINC,730.22,29.80,53.0,M-LC,10.67,211845.0,6769.0,105795.0,0.65,3.30,49.94,54.89,52.0,0.06,1.49,28.19,X5K,ATH,METALS
87,WIPRO,420.00,-13.34,50.0,M-LC,6.25,152334.0,1389.0,108066.0,1.80,0.92,70.94,72.51,53.0,0.01,1.07,7.00,XR,NTT,IT
37,INDIAMART,4810.62,-55.39,42.0,H-SC,7.36,124497.0,1161.0,130460.0,-1.17,0.94,104.79,106.72,119.0,0.01,0.88,24.54,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,17.0,X-LC,13.39,147995.0,-128528.0,238598.0,1.46,-46.48,161.22,39.81,54.0,-0.54,1.04,4.31,XY24,NTT,AUTO
53,QUESS,986.00,-48.21,36.0,X-SC,38.76,51684.0,-13322.0,164252.0,-1.15,-20.49,317.80,232.18,198.0,-0.08,0.36,0.81,XY24,NTT,MISC
24,DMART,5391.45,-15.93,39.0,X-LC,10.18,102874.0,288.0,26521.0,0.28,0.28,25.78,26.13,38.0,0.01,0.72,25.92,X40N,ATH,FMCG
33,HONAUT,58357.33,-23.25,44.0,X-SC,2.56,108195.0,-19743.0,66875.0,-1.08,-15.43,61.81,36.84,143.0,-0.30,0.76,10.77,X40N,ATH,ELECTRICAL
2,ABBOTINDIA,35195.00,-8.83,45.0,X-MC,3.49,90060.0,-618.0,15526.0,-0.78,-0.68,17.24,16.44,101.0,-0.04,0.63,19.36,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-1.97,50.0,H-LC,0.68,157500.0,-28595.0,76072.0,1.52,-15.37,48.30,25.51,15.0,-0.38,1.11,16.07,AR,ATH,ELECTRICAL
42,ITC,452.00,-37.08,63.0,X-LC,2.14,203351.0,3213.0,17671.0,0.73,1.61,8.69,10.44,4.0,0.18,1.43,7.26,X40,NTT,FMCG
85,VOLTAS,1530.00,2.56,59.0,X-MC,2.40,215250.0,23508.0,14250.0,-1.43,12.26,6.62,19.69,99.0,1.65,1.52,19.89,XY25,NTT,AC
29,HAVELLS,2069.16,-5.75,47.0,X-MC,2.81,233485.0,-14246.0,89308.0,0.85,-5.75,38.25,30.30,92.0,-0.16,1.65,6.78,X40,ATH,ELECTRICAL
58,RELIANCE,1533.00,-8.78,73.0,X-LC,3.06,228587.0,18221.0,10561.0,0.01,8.66,4.62,13.68,37.0,1.73,1.61,26.62,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.0,-48.21,36.0,X-SC,38.76,51684.0,-13322.0,164252.0,-1.15,-20.49,317.80,232.18,198.0,-0.08,0.36,0.81,XY24,NTT,MISC
3,ACC,3906.0,-38.07,47.0,X-MC,3.03,184000.0,-53951.0,206595.0,-0.48,-22.67,112.28,64.15,174.0,-0.26,1.30,3.21,XY24,BTT,CEMENT
74,TATAMOTORS,1065.0,-53.17,17.0,X-LC,13.39,147995.0,-128528.0,238598.0,1.46,-46.48,161.22,39.81,54.0,-0.54,1.04,4.31,XY24,NTT,AUTO
10,BAJAJHFL,181.5,-14.06,48.0,X-MC,5.67,181220.0,-19286.0,116434.0,-0.18,-9.62,64.25,48.45,90.0,-0.17,1.28,4.50,X40N,ATH,FINANCE
1,ABB,7934.0,-40.45,48.0,H-LC,6.50,248880.0,-12739.0,131956.0,-1.05,-4.87,53.02,45.57,7.0,-0.10,1.75,5.90,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,36.0,X-SC,38.76,51684.0,-13322.0,164252.0,-1.15,-20.49,317.80,232.18,198.0,-0.08,0.36,0.81,XY24,NTT,MISC
51,PAGEIND,51769.93,-25.96,46.0,X-MC,6.67,83590.0,80.0,19953.0,0.50,0.10,23.87,23.98,82.0,0.00,0.59,5.92,X40,ATH,APPARELS
57,RELAXO,1176.00,-38.72,48.0,X-SC,4.74,83481.0,-61679.0,137610.0,0.33,-42.49,164.84,52.31,136.0,-0.45,0.59,11.74,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-8.83,45.0,X-MC,3.49,90060.0,-618.0,15526.0,-0.78,-0.68,17.24,16.44,101.0,-0.04,0.63,19.36,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-34.72,50.0,X-SC,4.28,97575.0,-31095.0,76391.0,0.87,-24.17,78.29,35.20,219.0,-0.41,0.69,11.99,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-27.40,59.0,X-LC,12.50,289764.0,-56192.0,122889.0,2.52,-16.24,42.41,19.28,1.0,-0.46,2.04,7.12,X40,ATH,IT
40,INFY,2275.00,-15.65,61.0,X-LC,7.34,328318.0,15430.0,156247.0,4.69,4.93,47.59,54.87,3.0,0.10,2.31,11.91,X40,BTT,IT
42,ITC,452.00,-37.08,63.0,X-LC,2.14,203351.0,3213.0,17671.0,0.73,1.61,8.69,10.44,4.0,0.18,1.43,7.26,X40,NTT,FMCG
84,VBL,671.64,-16.88,58.0,X-LC,10.11,299222.0,-16620.0,129294.0,2.08,-5.26,43.21,35.67,5.0,-0.13,2.11,7.86,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,48.0,H-LC,6.50,248880.0,-12739.0,131956.0,-1.05,-4.87,53.02,45.57,7.0,-0.10,1.75,5.90,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7227.78,71.0,L-SC,7.00,80375.0,-13435.0,90052.0,0.22,-14.32,112.04,81.67,269.0,-0.15,0.57,55.65,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,26.64,81216.0,-32333.0,72388.0,-1.74,-28.47,89.13,35.27,268.0,-0.45,0.57,98.78,XR,NTT,HOTELS
49,MASFIN,398.61,-17.34,57.0,H-SC,9.56,93990.0,-3990.0,25593.0,1.69,-4.07,27.23,22.05,152.0,-0.16,0.66,36.62,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-38.27,67.0,M-SC,3.99,104280.0,12782.0,65968.0,-0.51,13.97,63.26,86.07,223.0,0.19,0.73,51.43,XR,NTT,DURABLES
46,KPIGREEN,731.05,4.41,61.0,H-SC,5.31,124375.0,-922.0,59849.0,-2.29,-0.74,48.12,47.03,141.0,-0.02,0.88,55.73,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-190.91,75.0,M-SC,22.62,157500.0,8280.0,135907.0,-0.07,5.55,86.29,96.62,208.0,0.06,1.11,70.45,XY25,NTT,FINANCE
79,UJJIVANSFB,60.00,122.87,69.0,H-SC,14.44,140508.0,-1971.0,21498.0,3.58,-1.38,15.30,13.70,163.0,-0.09,0.99,65.84,OX40N,NTT,BANKS
68,SONACOMS,806.63,-32.46,70.0,M-SC,10.22,83991.0,-17168.0,57173.0,3.90,-16.97,68.07,39.54,202.0,-0.30,0.59,18.44,AR,ATH,AUTO
5,ANGELONE,3033.00,16.33,70.0,X-SC,7.05,205697.0,14691.0,43011.0,0.43,7.69,20.91,30.21,157.0,0.34,1.45,30.18,X40N,NTT,FINANCE
59,REPCOHOME,880.00,-55.29,71.0,H-SC,2.51,255976.0,-30529.0,287871.0,1.67,-10.66,112.46,89.82,134.0,-0.11,1.80,35.43,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.53
1,25,43.93
2,50,74.78


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.32
LC    32.85
MC    22.82
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.30
X40      16.05
XY25     12.29
XR       11.27
X40N      9.32
AR        7.89
OX40N     7.62
X200      1.79
X5K       1.49
SR        1.09
MH        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.54
X-LC    22.00
X-MC    17.09
M-SC    12.68
M-LC     5.17
H-LC     4.65
X-SC     4.61
H-MC     3.85
M-MC     1.57
L-SC     1.49
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.78,-2.66,36.16
IT,12.77,-15.84,78.37
FINANCE,12.15,-9.70,59.43
MISC,6.87,-17.70,82.92
BANKS,6.62,-6.38,62.35
PAINTS,5.63,-15.59,32.82
ELECTRICAL,5.27,-10.07,48.69
INSURANCE,3.89,-0.06,37.36
AUTO,2.82,-43.42,103.27


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3275191.0,22
XR,1309977.0,14
AR,1173837.0,9
X40,764681.0,12
X40N,606007.0,8
OX40N,566253.0,10
XY25,439604.0,8
SR,266287.0,2
X5K,105795.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3377717.0,25
M-SC,1205907.0,15
X-MC,1110920.0,13
X-LC,1013609.0,13
X-SC,525594.0,6
H-MC,333378.0,3
H-LC,278944.0,3
M-LC,274475.0,4
L-SC,250649.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1162891.0      6
           AR         842350.0      5
           XR         783814.0      7
M-SC       XY24       688116.0      6
X-MC       XY24       584263.0      4
X-LC       X40        439996.0      5
           XY24       304266.0      2
X-SC       X40N       284951.0      4
H-SC       SR         266287.0      2
           OX40N      262526.0      4
X-MC       X40        248294.0      6
H-LC       AR         208028.0      2
X-LC       X40N       204622.0      3
H-MC       XY24       179846.0      1
M-MC       XY24       169413.0      1
X-SC       XY24       164252.0      1
L-SC       XR         162440.0      2
X-MC       XY25       161929.0      2
M-SC       OX40N      145525.0      4
           XY25       135907.0      1
           AR         123459.0      2
X-MC       X40N       116434.0      1
M-SC       XR         112900.0      2
M-LC       XR         108066.0      1
           X5K        105795.0      1
L-SC       OX40N       88209.0      1
H-MC       XR          83539.0      1
X-SC       X40         76391.0      1
H-LC       X200        70916.0      1
H-MC       OX40N       69993.0      1
X-LC       XY25        64725.0      3
H-SC       MH          59849.0      1
L-MC       XR          59218.0      1
L-LC       XY25        38573.0      1
M-LC       XY25        38470.0      1
           XY24        22144.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
